# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Turaev cobraket

In [8]:
def cobracket(myw, verbose: false)
  
  #-----
  # 2つの div の組み合わせ全てについて、{odds: *,  }
  #-----
  
  monomials = Wcode.new(myw).cobracket(verbose: verbose, narrow: narrow).map do |mono|
    prewords = mono[:factors].map(&:narrow).map(&:to_word)
    words = prewords.map(&:cyclic_reduce)
    mono.merge({coeff: mono[:linking], words: words})
  end

  if verbose
    result = monomials
  else
    result = []
    while monomials.size > 0
      pop = monomials.pop
      pop[:odds] = [pop[:odds]]
      #---
      monomials.each do |mono|
        if mono[:words][0].conjugate?(pop[:words][0]) and mono[:words][1].conjugate?(pop[:words][1])
          pop[:coeff] += mono[:coeff]
          pop[:odds] << mono[:odds]
          mono[:coeff] = nil
        elsif mono[:words][0].conjugate?(pop[:words][1]) and mono[:words][1].conjugate?(pop[:words][0])
          pop[:coeff] += mono[:coeff]*(-1)
          pop[:odds] << mono[:odds]
          mono[:coeff] = nil
        end
      end
      #---
      if pop[:coeff] != 0 or verbose
        result << pop
      end
      #---
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end 
  end
  return (result.empty?) ? [{orders: [], linking: 0, factors: [], coeff: 0, words: []}] : result
end; nil

# Execution

In [15]:
a1 = Word.new(a[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^4,
    c,
    comms[1]
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2])
  ]
  }

arr = samples.slice(:s6) #:s2) #, :s3) #:s0, :s1)
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each do |s|
    ws = s.show
    cob = cobracket(s).map do |h|
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      coeff + "#{h[:words].join(" \u{2227} ")}"
    end
    #---
    puts "#{ws}  |--\u{03B4}-->  #{cob.join(' + ')}"
  end
end;nil

--- s6 ---
 a.bcBC  |--δ-->  (-1)abcB ∧ C + abc ∧ BC + (-1)abC ∧ cB


In [10]:
myw = samples[:s6][-1]
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :odds, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

"a.bcBC"
"0|3,6|1|7,2|9,4||5,8|||||"
=====
"odds: [1, 3]"
"linking: 0"
"0|4|1|5|7,2||3,6||||| , |1||0||||||||"
"(0)acBC ∧ b"
----------
"odds: [1, 5]"
"linking: 0"
"0|2|1|3|5||4||||| , |1||0|2||3|||||"
"(0)aBC ∧ bc"
----------
"odds: [1, 7]"
"linking: -1"
"0||1||3||2||||| , |1,4||5,0|2||3|||||"
"(-1)aC ∧ c"
----------
"odds: [1, 9]"
"linking: 0"
"0||1||||||||| , |1,4||5,0|7,2||3,6|||||"
"(0)a ∧ bcBC"
----------
"odds: [3, 5]"
"linking: 1"
"0|3,4|1|5,2|7||6||||| , ||||0||1|||||"
"aC ∧ c"
----------
"odds: [3, 7]"
"linking: -1"
"0|3|1|2|5||4||||| , |2||3|0||1|||||"
"(-1)abC ∧ cB"
----------
"odds: [3, 9]"
"linking: -1"
"0|3|1|2|||||||| , |2||3|5,0||1,4|||||"
"(-1)ab ∧ B"
----------
"odds: [5, 7]"
"linking: 1"
"0|3|1|2|7,4||5,6||||| , |0||1||||||||"
"ab ∧ B"
----------
"odds: [5, 9]"
"linking: 1"
"0|3|1|2|4||5||||| , |0||1|3||2|||||"
"abc ∧ BC"
----------
"odds: [7, 9]"
"linking: -1"
"0|3,6|1|7,2|4||5||||| , ||||1||0|||||"
"(-1)abcB ∧ C"
----------


## Experiments

In [11]:
ws = [a1*(comms[2].inverse), a1*(c.inverse)*(comms[2].inverse)*c]
wcs = ws.map{|w| Wcode.new(w)}
wcs.each{|wc| p "#{wc.word.show} --> #{wc.show(3)}" }

cobras = ws.map{|w| cobracket(w, verbose: true, narrow: false)} # false)} #
printf "\n\n"
#---

(1..ws[0].size).map{|n| 2*n-1}.combination(2).each do |op|
  op2 = op.map{|o| (o == 1) ? o : o+6 }
  c0 = cobras[0].select{|h| h[:odds]==op}
  c1 = cobras[1].select{|h| h[:odds]==op2}

[c0, c1].each do |c|
  c.each do |h|
    h.each do |k,v|
      case k 
      when :odds, :linking
        p "#{k}: #{v}"
      when :factors
        p h[:factors].map{|f| f.show(3)}.join(" , ")
      when :words
        coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
        p coeff + "#{h[:words].join(" \u{2227} ")}"
      else
      end
    end
    printf("----------\n")
  end;
end
  printf("=================\n")
end; nil

"a.dcDC --> (0||1|)(9,4|3,6|5,8|7,2)(|||)"
"a.aBA.dcDC.abA --> (21,7,0,2,16|19,4|17,3,1,6,20|5,18)(15,10|9,12|11,14|13,8)(|||)"


"odds: [1, 3]"
"linking: 0"
"(0||1|)(9,4|6|5,8|7)(|||) , (|)(|)(|3)(|2)(|)(|)"
"(0)acDC ∧ d"
----------
"odds: [1, 9]"
"linking: 0"
"(21,0,16|19|17,1,20|18)(15,10|12|11,14|13)(|||) , (7,2|4|3,6|5)(|9||8)(|||)"
"(0)cDCab ∧ aBAd"
----------
"odds: [1, 5]"
"linking: 0"
"(0||1|)(9|6|8|7)(|||) , (|||)(4|3|5|2)(|||)"
"(0)aDC ∧ dc"
----------
"odds: [1, 11]"
"linking: 0"
"(21,0,16|19|17,1,20|18)(15|12|14|13)(|||) , (7,2|4|3,6|5)(10|9|11|8)(|||)"
"(0)DCab ∧ aBAdc"
----------
"odds: [1, 7]"
"linking: 0"
"(0||1|)(9||8|)(|||) , (|||)(4|3,6|5|7,2)(|||)"
"(0)aC ∧ c"
----------
"odds: [1, 13]"
"linking: 0"
"(21,0,16|19|17,1,20|18)(15||14|)(|||) , (7,2|4|3,6|5)(10|9,12|11|13,8)(|||)"
"(0)Cab ∧ aBAdcD"
----------
"odds: [1, 9]"
"linking: -1"
"(0|)(1|)(|)(|)(|)(|) , (|||)(9,4|3,6|5,8|7,2)(|||)"
"(-1)a ∧ dcDC"
----------
"odds: [1, 15]"
"linking: -1"
"(21,0,16|19|17,1,20|18)(

In [12]:
myw = a1*(comms[2].inverse)
#myw = a1*Group.commutator(b[1], a[2]) #
#myw = a1*Group.conjugate(Group.commutator(b[1], a[2]), c)
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :odds, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

"a.dcDC"
"0||1||9,4|3,6|5,8|7,2||||"
=====
"odds: [1, 3]"
"linking: 0"
"0||1||7,2|4|3,6|5|||| , |||||1||0||||"
"(0)acDC ∧ d"
----------
"odds: [1, 5]"
"linking: 0"
"0||1||5|2|4|3|||| , ||||2|1|3|0||||"
"(0)aDC ∧ dc"
----------
"odds: [1, 7]"
"linking: 0"
"0||1||3||2||||| , ||||2|1,4|3|5,0||||"
"(0)aC ∧ c"
----------
"odds: [1, 9]"
"linking: -1"
"0||1||||||||| , ||||7,2|1,4|3,6|5,0||||"
"(-1)a ∧ dcDC"
----------
"odds: [3, 5]"
"linking: 0"
"0||1||7|3,4|6|5,2|||| , ||||0||1|||||"
"(0)aC ∧ c"
----------
"odds: [3, 7]"
"linking: 0"
"0||1||5|3|4|2|||| , ||||0|2|1|3||||"
"(0)adC ∧ cD"
----------
"odds: [3, 9]"
"linking: 0"
"0||1|||3||2|||| , ||||5,0|2|1,4|3||||"
"(0)ad ∧ D"
----------
"odds: [5, 7]"
"linking: 0"
"0||1||7,4|3|5,6|2|||| , |||||0||1||||"
"(0)ad ∧ D"
----------
"odds: [5, 9]"
"linking: 0"
"0||1||4|3|5|2|||| , ||||3|0|2|1||||"
"(0)adc ∧ DC"
----------
"odds: [7, 9]"
"linking: 0"
"0||1||4|3,6|5|7,2|||| , ||||1||0|||||"
"(0)adcD ∧ C"
----------


In [13]:
p myw.show
d1 = (a1.size)+1
d2 = cif + c.size
d3 = myw.size - (c.size-1)
d4 = myw.size + 1

r1 = (d1..d2-1).map{|n| 2*n-1}.to_a
r2 = (d3..d4-1).map{|n| 2*n-1}.to_a
p r1, r2
p r1.include?(9)
nil

"a.dcDC"


NameError: undefined local variable or method `cif' for main:Object

### TODO

* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。
* [ ] a.abA.aBA の cobraket にバグあり。
* [ ] div 同士の linking を div に対応する2文字ワード同士の単純な計算で求められないか?
* [ ] Wcode#devide して得られる2つの Wcode instance の show(2) (および show(3)) がバグってる。なぜか each_slice(4) が2つずつくくる。
* [x] 係数がプラマイ逆かな。
* [x] simple subword たちによる計算を実装する。<-- ダメだった。各 simple subword が内部では確実に交差しない保証がないとダメだから。